In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
from tqdm.notebook import tqdm
from itertools import chain
from bs4 import BeautifulSoup
import time
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
import pymorphy2
import functools
from sklearn.metrics import f1_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pymorphy2
import random
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from nltk.corpus import stopwords
import scipy
import copy
from sklearn.gaussian_process import GaussianProcessClassifier
# import gensim 

# model2=gensim.models.KeyedVectors.load("model.model")
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words(["russian", "english"]))
stemmerR = SnowballStemmer("russian")
stemmerE = SnowballStemmer("english")

# Перевод слов

In [25]:
import requests
lit = set('abcdefghijklmnopqrstuvwxyz')

def translate_yandex(text):
    if text[0] in lit:
        LANG='en-ru'
    else:
        return text
    URL = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
    KEY = 'trnsl.1.1.20160119T035517Z.50c6906978ef1961.08d0c5ada49017ed764c042723895ffab867be7a'
    TEXT = text
 
    r = requests.post(URL, data={'key': KEY, 'text': TEXT, 'lang': LANG})
    return r.text[r.text.find('['):-3][2:]

'машина'

# Считывание заголовков

In [2]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [3]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', "r", "utf_8_sig") as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [4]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()
         if not word in stop_words]
    vec[ID] = ' '.join([word for word in words if not word in stop_words])

In [5]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles, list(range(1,28027)))

1) Заголовки содержат очень много английских слов, которые мы хотели перевести. Стало хуже. Многие слова не имею перевода, так как являются странными названиями сайтов.

2) LDA на заголовках выдавал полную чушь.

3) Стемминг показал себя лучше лемматизации. Было использовано много видов лемматизации: Wordnet, pymorphy2, StenfordNLP.

# Парсим тексты

In [ ]:
# nltk.download()

In [35]:
def illustration(func):
    """
    Распаралеливание выкачки страниц.
    """
    mutex = ThreadLock()
    n_thread = ThreadValue('i',0)
    @functools.wraps(func)
    def wrapper(*args, **argv):
        result = func(*args, **argv)
        with mutex:
            nonlocal n_thread
            n_thread.value +=1
            print(f"\r{n_thread.value} objects are processed...",end ='',flush = True)
        return result
    return wrapper

In [19]:
@illustration
def Parse(num):
    lit=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ')
    file = 'texts/content/{}.dat'.format(num)
    f = codecs.open(file, encoding='utf-8')
    soup = BeautifulSoup(f, 'lxml')
    cont = soup.find_all('div')
    result = []
    for i in cont:
        a=i.get_text().strip()
        res=a.split()
        for i in res:
            fl=1
            for j in i:
                if not(j in lit):
                    fl=0
                    break
            if fl and not i.lower() in stop_words:
                result.append(stemmerR.stem(i.lower()))
    doc_to_text[num] = ' '.join(result)

In [20]:
doc_to_text = {}
with ThreadPool(10) as pool: 
    pool.map(Parse, list(range(1,28027)))

28026 objects are processed...

In [25]:
def Parser_for_texts_without_stemming (texts_dict):
    words_dict = {}
    for ID, text in tqdm(texts_dict.items()):
        words_dict[ID] = ' '.join([word for word in text.split() if not word in stop_words])
    return words_dict

In [26]:
doc_to_text = Parser_for_texts_without_stemming(doc_to_text)

In [22]:
doc_txt_list = list(doc_to_text.values())
df = pd.DataFrame(data = {'txt': doc_txt_list}, index=doc_to_text.keys())
df['ind']=df.index
df = df.sort_values('ind')
df = df.drop(['ind'], axis=1)
df.shape

(28026, 1)

In [ ]:
df.to_csv('text_result_plus_stemm.csv', encoding = 'utf_8_sig')

In [ ]:
doc_to_text = {}
with codecs.open('texts/text_result_plus_stemm.csv', "r", 'utf_8_sig') as f:
    next(f)
    for line in tqdm(f):
        data = line.strip().split(',', 1)
        doc_to_text[int(data[0])] = data[1]


print(len(doc_to_text))

In [ ]:
#Можно просто так.
a = pd.read_csv('texts/text_result_plus_stemm.csv')
doc_to_text={}
for i in range(1, 28027):
    doc_to_text[i]=a.txt[i-1]

In [ ]:
# def Parser_for_text(text_dict):
#     words_dict = {}
#     for ID, text in tqdm(text_dict.items()):
#         words_dict[ID] = text.split(' ')
#     return words_dict

# vec_txt = Parser_for_text(doc_to_text)

1) LDA на текстах выдал полный бред

2) Score для текстов в разы меньше

3) также пробовали выделять заголовки h1, h2, ..., h6 на сайтах. Получалось плохо. Было много "пустых" заголовков.

4) Пробовали использовать количество картинок, видео, скрипта для введения доп. фичей. Score только падал.

# Фичи

1) cos фичи

2) jac фичи

3) dbscan - поиск аномалий

4) Seva

5) Пытались вытащить много со страниц, но не помогло

6) Пытались добавить семантические расстояния, но тоже слабовато

In [6]:
class Importer:

    def __init__(self, doc_to_title, n_f, nfj=0):
        doc_to_title[0] = ''
        self.doc_to_vec = TfidfVectorizer().fit_transform([doc_to_title[i] for i in range(len(doc_to_title))])
        dtt={}
        for i in doc_to_title.keys():
            dtt[i]=doc_to_title[i].split(' ')
        self.doc = dtt
        self.n_features = n_f
        self.n_features_jac = nfj
        self.extract_features = self.cosine
   
    def cosine(self, group):
        
        def jaccard(group):
            X = np.empty(shape=(group.size, self.n_features_jac), dtype=np.float)
            for i, doc_ID in enumerate(group):
                dist = np.empty(shape=(group.size, ), dtype=np.float)
                vec = set(self.doc[doc_ID])
                for j, doc_j in enumerate(group):
                    if doc_ID == doc_j:
                        dist[j] = 0
                    else:
                        vec_j = set(self.doc[doc_j])
                        intersec = len(vec.intersection(vec_j))
                        if intersec > 0:
                            dist[j] = intersec/(len(vec) + len(vec_j) - intersec)
                        else:
                            dist[j] = 0
                X[i] = np.sort(dist)[-self.n_features_jac:]
            return X
    
        n = (self.n_features)//2
        X = np.empty(shape=(group.size, self.n_features), dtype=np.float)
        for i, title in enumerate(pairwise_distances(self.doc_to_vec[group], metric='cosine')):
            X[i, :n] = sorted(title)[1:n + 1]
        X[:, n:] = np.mean(X[:, :n], axis=0)
        X[:, :n] /= X[:, n:]
        if (self.n_features_jac==0):
            return X
        X_jac = jaccard(group)
        return np.hstack((X,X_jac))
    
           
    def import_from(self, file):
        data = pd.read_csv(file)
        groups = data.groupby('group_id')
        if 'target' in data.columns:
            X = np.empty(shape=(data.shape[0], self.n_features + self.n_features_jac +1), dtype=np.float)
            y = np.empty(shape=(data.shape[0], ), dtype=bool)
            group_ids = np.empty(shape=(data.shape[0], ), dtype=int)
            i = 0
            for group_id, group_indx in groups.groups.items():
                j = i + group_indx.size
                group = data.iloc[group_indx]
                group_ids[i:j] = group_id
                clf = DBSCAN(0.495, metric="cosine", min_samples=7)
                res = clf.fit_predict(self.doc_to_vec[group.doc_id])
                for t in range(len(res)):
                    if res[t] == -1:
                        res[t] = 0
                    else:
                        res[t] = 1
                y[i:j] = group.target
                X[i:j,:-1] = self.extract_features(group.doc_id)
                X[i:j,-1] = res
                i = j
 
            return X, y, group_ids
        else:
            X = np.empty(shape=(data.shape[0], self.n_features + self.n_features_jac +1), dtype=np.float)
            pair_ids = np.empty(shape=(data.shape[0], ), dtype=int)
 
            i = 0
            for group_id, group_indx in groups.groups.items():
                j = i + group_indx.size
                group = data.iloc[group_indx]
                pair_ids[i:j] = group.pair_id
                clf = DBSCAN(0.495, metric="cosine", min_samples=7)
                res = clf.fit_predict(self.doc_to_vec[group.doc_id])
                for t in range(len(res)):
                    if res[t] == -1:
                        res[t] = 0
                    else:
                        res[t] = 1
                X[i:j,:-1] = self.extract_features(group.doc_id)
                X[i:j,-1] = res
                i = j
 
            return X, pair_ids

In [7]:
#СЕВА, СПАСБО ЗА ФИЧИ!
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
    
y_train_SEV = []
X_train_SEV = []
groups_train_SEV = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train_SEV.append(target_id)
        groups_train_SEV.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train_SEV.append(sorted(all_dist, reverse=True)[0:15]    )
X_train_SEV = np.array(X_train_SEV)
y_train_SEV = np.array(y_train_SEV)
groups_train_SEV = np.array(groups_train_SEV)
print (X_train_SEV.shape, y_train_SEV.shape, groups_train_SEV.shape)

test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))
    
X_test_SEV = []
groups_test_SEV = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test_SEV.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test_SEV.append(sorted(all_dist, reverse=True)[0:15]    )
X_test_SEV = np.array(X_test_SEV)
groups_test_SEV = np.array(groups_test_SEV)
print (X_test_SEV.shape, groups_test_SEV.shape)

scale_features_std = StandardScaler()
X_train_SEV=scale_features_std.fit_transform(X_train_SEV)
X_test_SEV=scale_features_std.fit_transform(X_test_SEV)
print(X_train_SEV.shape, X_test_SEV.shape)

(11690, 15) (11690,) (11690,)
(16627, 15) (16627,)
(11690, 15) (16627, 15)


# Валидации (малая её часть)

1) Перебрали, наверное, все модели sklearn, Чуть меньше 40 моделей валидировали.

2) Подбирали параметры модели и фичи к ней.

3) СТранно, но обычное разбиение по фолдам - не очень хорошо себя показало. Лучше было рандомное разбиение по фолдам (shuffle=True). Хотя под конец уже начали сомневаться в этом.

In [51]:
def cross_val_gauss():
    cos_f = range(2,30,2)
    jac_f = range(0, 7)
    dbscan_f = [0, 1]
    SEV_f = [0, 15]
    for c_f in tqdm(cos_f):
        for j_f in jac_f:
            for db_f in dbscan_f:
                for S_f in SEV_f:
                    importer = Importer(vec, c_f, j_f)
                    X, y, groups_train = importer.import_from('train_groups.csv')
                    scaler = StandardScaler()
                    scaler.fit(X[:, :c_f+j_f])
                    if db_f==1:
                        lft = scaler.transform(X[:,:c_f+j_f])
                        X = np.hstack((lft, X[:,c_f+j_f][:,np.newaxis]))
                    else:
                        X = scaler.transform(X[:,:c_f+j_f])
                    X = np.hstack((X, X_train_SEV[:,:S_f]))
                    model = GaussianProcessClassifier(n_jobs = -1)
                    print([cross_val_score(model, X, y, cv=10,scoring = 'f1').mean(),c_f, j_f, db_f, S_f, X.shape])
                    
cross_val_gauss()

In [ ]:
def cross_val_knn():
    cos_f = range(2,30,2)
    jac_f = range(0, 7)
    dbscan_f = [0, 1]
    SEV_f = [0, 15]
    nebrs = range(20, 220, 5)
    f1 = []
    for c_f in tqdm(cos_f):
        for j_f in jac_f:
            for db_f in dbscan_f:
                for S_f in SEV_f:
                    importer = Importer(vec, c_f, j_f)
                    X, y, groups_train = importer.import_from('train_groups.csv')
                    scaler = StandardScaler()
                    scaler.fit(X[:, :c_f+j_f])
                    if db_f==1:
                        lft = scaler.transform(X[:,:c_f+j_f])
                        X = np.hstack((lft, X[:,c_f+j_f][:,np.newaxis]))
                    else:
                        X = scaler.transform(X[:,:c_f+j_f])
                    X = np.hstack((X, X_train_SEV[:,:S_f]))
                    for n in nebrs:
                        model = KNeighborsClassifier(n)
                        
                        f1.append([cross_val_score(model, X, y, cv=10,scoring = 'f1', n_jobs=-1).mean(),c_f, j_f, db_f, S_f, X.shape, n])
    return f1
                    
f = cross_val_knn()

In [23]:
def cros_val_neural(X,y,act = 'tanh',cv = 10):
    n_hidden = np.arange(5,205,10)
    n_layers = [1, 2]
    n_s = 0
    error = []
    for n_l in n_layers:
        for n_h in tqdm(n_hidden):
            if(n_l == 1):
                kf = KFold(n_splits=cv,shuffle=True)
                model = MLPClassifier(max_iter=800, hidden_layer_sizes=(n_h,), random_state=1, activation=act)
                e = cross_val_score(model, X, y, cv=kf,scoring = 'f1',n_jobs = -1).mean()
                print(e,n_l,n_h,n_s)
                error.append([e,n_l,n_h,n_s])
            else:
                for n_h_2 in n_hidden:
                    kf = KFold(n_splits=cv,shuffle=True)
                    model = MLPClassifier(max_iter=800, hidden_layer_sizes=(n_h,n_h_2), random_state=1, activation=act)
                    e = cross_val_score(model, X, y, cv=kf,scoring = 'f1',n_jobs = -1).mean()
                    print(e,n_l,n_h,n_h_2)
                    error.append([e,n_l,n_h,n_h_2])
    return error


In [ ]:
# 0.752874777765091 2 60 10, 0.7503708319809625 2 10 10 для relu
#0.7557752644516544 2 10 80, 0.7550001382383368 2 15 10 для tanh
import warnings
warnings.filterwarnings("ignore")

cc = 20
cj = 5
importer = Importer(vec, cc, cj)
X_train, y_train, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X_train[:,:cc+cj])
X_train = scaler.transform(X_train[:,:cc+cj])
err = cros_val_neural(X_train, y_train)

# Обучение

In [53]:
# data_train = pd.read_csv('train_groups.csv')
# groups_train = data_train.groupby('group_id')
# data_test = pd.read_csv('test_groups.csv')
# groups_test = data_test.groupby('group_id')
# groups = {}
# for ID, group_indx in groups_train.groups.items():
#     group = data_train.iloc[group_indx]
#     groups[ID] = group.doc_id.values
    
# for ID, group_indx in groups_test.groups.items():
#     group = data_test.iloc[group_indx]
#     groups[ID] = group.doc_id.values

309

In [8]:
# def best_th(cls, X12, X3, X4, X56, X7, y, groups, k=5):
#     space = np.linspace(0.1, 0.9, 25)
#     result = np.zeros_like(space)
#     for train, test in GroupKFold(n_splits=k).split(X12, y, groups):
#         cls.fit(X12[train], X3[train], X4[train],X56[train],X7[train], y[train])
#         predict = cls.predict(X12[test], X3[test], X4[test], X56[test], X7[test])
        
#         for i, th in enumerate(space):
#             result[i] += f1_score(y[test], predict > th)

#     best = np.argmax(result)
#     return space[best], result[best] / k

In [8]:
importer = Importer(vec, 8, 3)
X12, y, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X12[:,:11])
lft = scaler.transform(X12[:,:11])
X12 = np.hstack((lft, X12[:,11][:,np.newaxis]))

importer = Importer(vec, 26, 4)
X3, y, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X3[:,:30])
X3 = scaler.transform(X3[:,:30])

importer = Importer(vec, 8, 3)
X4, y, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X4[:,:11])
lft = scaler.transform(X4[:,:11])
X4 = np.hstack((lft, X4[:,11][:,np.newaxis]))

importer = Importer(vec, 20, 5)
X56, y, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X56[:,:25])
X56 = scaler.transform(X56[:,:25])

importer = Importer(vec, 8, 3)
X7, y, groups_train = importer.import_from('train_groups.csv')
scaler = StandardScaler()
scaler.fit(X7)
X7 = scaler.transform(X7)
X7 = np.hstack((X7, X_train_SEV))
X_TR = [X12, X3, X4, X56, X7]

1) Регрессия показала себя намного хуже обычного голосования, при этом для неё ещё нужно было подбирать th.

2) Пробовали делать миллиарды стеккингов с всевозможными внешними функциями, а также объединять в голосование их... Плохо... Переобучение огромное. Валидация выдавала 79.2-79.7 на всех 9 моделях. Голосование из них выдало 0.8. Но на каггле было 74.2. А времени убили колоссальное количество.

3) Обычное голосование показало себя превосходно.

4) Обычная нейронка с тангенсальной функцией активации тоже довольно стабильна и просто в исполнении + неплохой скор.

In [9]:
# #Регрессия
# class combined_cls:
#     def __init__(self):
# #         self.cls1 = LogisticRegression(solver='lbfgs', max_iter=1000, C=20.0) #все фичи (8, 3, 1, Seva)
#         self.cls2 = LinearRegression() #все фичи (8, 3, 1, Seva)
#         self.cls3 = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=0, n_jobs=-1) #26 соседей cos
# #         self.cls4 = GaussianNB () #12 cos, 3 jac, dbscan
#         self.cls5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=1) #20 соседей cos
#         self.cls6 = MLPClassifier(max_iter=800, hidden_layer_sizes=(5, 5), random_state=1, activation='tanh') #20 соседей cos
#         self.cls7 = SVC(C = 0.58,kernel = 'rbf', gamma='auto') #20 cos, 5 jac, Seva
#         self.cls = LinearRegression()
        
#     def fit(self, X12, X3, X4, X56, X7, y):

# #         self.cls1.fit(X12, y)
#         self.cls2.fit(X12, y)
#         self.cls3.fit(X3, y)
# #         self.cls4.fit(X4, y)
#         self.cls5.fit(X56, y)
#         self.cls6.fit(X56, y)
#         self.cls7.fit(X7, y)
        
#         self.cls.fit(np.array([
# #             self.cls1.predict_proba(X12)[:, 1],
#             self.cls2.predict(X12), 
#             self.cls3.predict_proba(X3)[:, 1], 
# #             self.cls4.predict_proba(X4)[:, 1],
#             self.cls5.predict_proba(X56)[:, 1],
#             self.cls6.predict_proba(X56)[:, 1],
#             self.cls7.predict(X7)
#         ]).T, y)
        
#     def predict(self, X12, X3, X4, X56, X7):
#         return self.cls.predict(np.array([
# #             self.cls1.predict_proba(X12)[:, 1],
#             self.cls2.predict(X12), 
#             self.cls3.predict_proba(X3)[:, 1],
# #             self.cls4.predict_proba(X4)[:, 1],
#             self.cls5.predict_proba(X56)[:, 1],
#             self.cls6.predict_proba(X56)[:, 1],
#             self.cls7.predict(X7)
#         ]).T)

#74.5 на Kaggle

In [9]:
from sklearn.base import BaseEstimator
class Voiting(BaseEstimator):
    def __init__(self):
        #self.cls1 = Linear(0.34) #все фичи (8, 3, 1, Seva)
        self.cls2 = KNeighborsClassifier(42,weights='distance',p = 1)
        self.cls3 = RandomForestClassifier(n_estimators=600, max_depth=10,criterion='entropy', random_state=0, n_jobs=-1) #26 соседей cos
    #    self.cls4 = GaussianNB () #12 cos, 3 jac, dbscan
        self.cls5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=1) #20 соседей cos
        self.cls6 = MLPClassifier(max_iter=800, hidden_layer_sizes=(5, 5), random_state=1, activation='tanh') #20 соседей cos
        self.cls7 = SVC(C = 483.2930238571752,kernel = 'rbf', gamma='auto') #20 cos, 5 jac, Seva
     #   self.cls8 = GaussianProcessClassifier(n_jobs=-1)
    def fit(self,X,y):
        X12, X3, X4, X56, X7 = X
        self.cls2.fit(X12, y)
        self.cls3.fit(X3, y)
       # self.cls4.fit(X4, y)
        self.cls5.fit(X56, y)
        self.cls6.fit(X56, y)
        self.cls7.fit(X7, y)
     #   self.cls8.fit(X7, y)
    def predict(self,X):
        X12, X3, X4, X56, X7 = X
        self.pred = np.transpose(np.array([self.cls2.predict(X12),
                              self.cls3.predict(X3),
                                # self.cls4.predict(X4),
                                self.cls5.predict(X56),
                                self.cls6.predict(X56),
                                self.cls7.predict(X7)]))#,
                         #       self.cls8.predict(X7)]))
        def pr(line):
            ar,count = np.unique(line,return_counts = True)
            return ar[np.argmax(count)]
        predict = np.array([pr(line) for line in self.pred])
        return predict    
    
#75.8 на Kaggle

In [31]:
# cls = MLPClassifier(max_iter=800, hidden_layer_sizes=(5, 5), random_state=1, activation='tanh')
# cls.fit(X56, y_train_SEV)

# predict = cls.predict(X56_test)

# 75.2 на Kaggle

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 80), learning_rate='constant',
              learning_rate_init=0.001, max_iter=800, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

# Предсказание

In [10]:
model = Voiting()
model.fit(X_TR,y)

In [11]:
importer = Importer(vec, 8, 3)
X12, pair_ids = importer.import_from('test_groups.csv')
scaler = StandardScaler()
scaler.fit(X12[:,:11])
lft = scaler.transform(X12[:,:11])
X12 = np.hstack((lft, X12[:,11][:,np.newaxis]))
# X12 = np.hstack((X12, X_test_SEV))

importer = Importer(vec, 26, 4)
X3, pair_ids = importer.import_from('test_groups.csv')
scaler = StandardScaler()
scaler.fit(X3[:,:30])
X3 = scaler.transform(X3[:,:30])

importer = Importer(vec, 8, 3)
X4, pair_ids = importer.import_from('test_groups.csv')
scaler = StandardScaler()
scaler.fit(X4[:,:11])
lft = scaler.transform(X4[:,:11])
X4 = np.hstack((lft, X4[:,11][:,np.newaxis]))

importer = Importer(vec, 20, 5)
X56, pair_ids = importer.import_from('test_groups.csv')
scaler = StandardScaler()
scaler.fit(X56[:,:25])
X56 = scaler.transform(X56[:,:25])

importer = Importer(vec, 8, 3)
X7, pair_ids = importer.import_from('test_groups.csv')
scaler = StandardScaler()
scaler.fit(X7)
X7 = scaler.transform(X7)
X7 = np.hstack((X7, X_test_SEV))
X_TE = [X12, X3, X4, X56, X7]

In [12]:
def save_result(pair_ids, predict, filename='predict.csv'):
    with open(filename, 'w') as f:
        print('pair_id,target', file = f)
        for pair_id, target in zip(pair_ids, predict):
            print(pair_id, int(target), sep=',', file=f)

In [13]:
predict = model.predict(X_TE)
save_result(pair_ids, predict, 'predict.csv')

EOF